In [0]:
import shutil
import os
import glob
from datetime import datetime, timedelta, timezone

import mlflow
from mlflow.tracking import MlflowClient
from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions
from azure.identity import ClientSecretCredential

class AzureConfig:

    def get_secret(self, key):
        return dbutils.secrets.get(scope="kv-databricks-learn", key=key)
    
    def __init__(self):
        self.storage_account = self.get_secret("storageaccount")
        self.azure_blob_container = self.get_secret("azureblobcontainer")
        self.models_folder = "models"
        self.application_id = self.get_secret("applicationid")
        self.directory_id = self.get_secret("directoryid")
        self.service_credential = self.get_secret("servicecredential")


def export_model_to_storage(model_name: str, model_version: str, config: AzureConfig):

    print('Getting storage credentials ...')

    blob_name = f"{config.models_folder}/{model_name}_{model_version}.zip"

    # --- AUTH (already configured in your message) ---

    spark.conf.set(f"fs.azure.account.auth.type.{config.storage_account}.dfs.core.windows.net", "OAuth")
    spark.conf.set(f"fs.azure.account.oauth.provider.type.{config.storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
    spark.conf.set(f"fs.azure.account.oauth2.client.id.{config.storage_account}.dfs.core.windows.net", config.application_id)
    spark.conf.set(f"fs.azure.account.oauth2.client.secret.{config.storage_account}.dfs.core.windows.net", config.service_credential)
    spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{config.storage_account}.dfs.core.windows.net", f"https://login.microsoftonline.com/{config.directory_id}/oauth2/token")

    client = MlflowClient(registry_uri="databricks")

    # --- DOWNLOAD MODEL FROM REGISTRY ---
    model_uri = f"models:/{model_name}/{model_version}"
    local_dir = 'model'
    os.makedirs(local_dir, exist_ok=True)
    mv = client.get_model_version(name=model_name, version=model_version)

    print(f"Downloading model from {model_uri}...")
    local_path = client.download_artifacts(run_id=mv.run_id, path=local_dir)


    artifact = f'{local_path}/artifacts'
    zip_path = f'{artifact}/{model_name}_{model_version}.zip'

    # print("Listing files in", artifact)
    # for file in glob.glob(f"{artifact}/**", recursive=True):
    #     print(file)

    print('zip_path', zip_path)


    # --- UPLOAD TO AZURE STORAGE ---
    print('Uploading to Azure Storage')

    target_blob_file = f"abfss://{config.azure_blob_container}@{config.storage_account}.dfs.core.windows.net/{blob_name}"
    print(f"Uploading to {target_blob_file}...")

    dbutils.fs.cp(f"file:{zip_path}", target_blob_file)

    print('UPLOADED')

    return blob_name

azure_config = AzureConfig()

model_name = "aisdk_model_state_identifier-edge"
model_version = 1

blob_name = export_model_to_storage(model_name, model_version, azure_config)
    


In [0]:

def generate_sas_token(blob_name: str, config: AzureConfig):

    credential = ClientSecretCredential(config.directory_id, config.application_id, config.service_credential)
    account_url=f"https://{config.storage_account}.blob.core.windows.net"

    blob_service_client = BlobServiceClient(
        account_url=account_url,
        credential=credential
    )

    # get a user delegation key
    udk = blob_service_client.get_user_delegation_key(
        key_start_time=datetime.now(timezone.utc),
        key_expiry_time=datetime.now(timezone.utc) + timedelta(hours=1)
    )

    # generate SAS
    sas_token = generate_blob_sas(
        account_name=config.storage_account,
        container_name=config.azure_blob_container,
        blob_name=blob_name,
        user_delegation_key=udk,
        permission=BlobSasPermissions(read=True, list=True),
        expiry=datetime.now(timezone.utc) + timedelta(hours=1)
    )

    sas_url = f"{account_url}/{config.azure_blob_container}/{blob_name}?{sas_token}"
    print("✅ SAS URL:", sas_url)

    return sas_url

generate_sas_token(blob_name, azure_config)